Conclusion: Cypher projection is much slower and below tests ran into errors
ClientError: ProcedureCallFailed: Failed to invoke procedure `gds.graph.create.cypher`: Caused by: java.lang.IllegalStateException: Failed to load relationship with unknown source-node id 2012406.


In [1]:
# Continue from here:
# https://tbgraph.wordpress.com/2020/03/09/exploring-the-graph-catalog-feature-of-neo4j-graph-data-science-plugin-on-a-lord-of-the-rings-dataset/

In [2]:
from py2neo import *
import pandas as pd

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [4]:
graph = Graph()

In [5]:
# Projecting entire graph into memory (~8GB)

In [6]:
# NOTE: neo4j memory configuration change to below values in:
# "F:\neo4j-data\neo4jDatabases\database-355f6508-a0f2-4632-9ac2-65096a4d3673\installation-3.5.19\conf\neo4j.conf"

# dbms.memory.heap.initial_size=6G
# dbms.memory.heap.max_size=8G

# dbms.memory.pagecache.size=2G

In [7]:
%%time

# GDS library needs to be installed:
# https://neo4j.com/docs/graph-data-science/current/installation/

# NOTE!!! Set to limit projection to Category nodes only!
'''
commandToRun = "CALL gds.graph.create.cypher \
                ('allCatsCyph_graph', \
                'MATCH (n:Category) RETURN id(n) AS id', \
                'MATCH (m)-[r:BELONGS_TO]->(o) RETURN id(m) AS source, id(o) AS target, type(r) as type')"

'''

commandToRun = "CALL gds.graph.create.cypher('allCatsCyph_graph','MATCH (n:Category) RETURN id(n) AS id','MATCH (m)-[r:BELONGS_TO]->(o) RETURN id(m) AS source, id(o) AS target, type(r) as type')"

graph.run(commandToRun)

ClientError: ProcedureCallFailed: Failed to invoke procedure `gds.graph.create.cypher`: Caused by: java.lang.IllegalStateException: Failed to load relationship with unknown source-node id 2012406.

In [ ]:
%%time
commandToRun = "CALL gds.wcc.stream('allCatsCyph_graph') YIELD nodeId, componentId \
                RETURN componentId, count(*) as size \
                ORDER BY size DESC LIMIT 10"
pd.DataFrame(graph.run(commandToRun).data())

In [ ]:
%%time
commandToRun = "CALL gds.wcc.stream('allCatsCyph_graph') YIELD nodeId, componentId \
                RETURN componentId, count(*) as size \
                ORDER BY size DESC"
pd.DataFrame(graph.run(commandToRun).data())

In [ ]:
%%time
commandToRun = "CALL gds.graph.list()"
pd.DataFrame(graph.run(commandToRun).data())

In [ ]:
%%time
commandToRun = "CALL gds.graph.drop('allCatsCyph_graph')"
graph.run(commandToRun)